In [3]:
import pandas as pd
import numpy as np
import mating
import newMultiPop
from createPop import *
from standardFunctions import wavg
import os
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CDSView, GroupFilter

def reshapeSingle(data):
    data["Gen"] = data["Gen"].astype('int64')
    parameters = data.Pheno.unique()
    Gens = data.Gen.unique()
    added = []
    dataList = []
    for par in parameters:
        parDF = data.loc[data['Pheno']==par]
        parDF = parDF[['Pop', 'Gen', 'Value']]
        parDF = parDF.groupby(['Pop', 'Gen']).Value.sum().to_frame()
        parDF.rename(index=str, columns={"Value": par}, inplace=True)
        dataList.append(parDF)
    
    dataReshape = dataList[0].join(dataList[1], on=["Pop", "Gen"])
    for df in dataList[2:]:
        dataReshape = dataReshape.join(df, on=["Pop", "Gen"])
    data = dataReshape

    data[["M", "F", "T", "migrations", "Matings", "Contacts", "MMContacts"]] = data[["M", "F", "T", "migrations", "Matings", "Contacts", "MMContacts"]].astype('Int64')
    totalPopDict = {}
    for col in data.select_dtypes('Int64').columns:
        totalPopDict[col] = data.groupby(["Gen"])[col].sum()

    for col in ["A", "I", "O", "FemF"]:
        totalPopDict[col] = data.groupby(["Gen"]).apply(wavg, col, "F")

    for col in ["preAPref", "preIPref", "preOPref", "APref", "IPref", "OPref", "MalF"]:
        totalPopDict[col] = data.groupby(["Gen"]).apply(wavg, col, "M")

    #totalPopDict["Run", "Gen"] = data.groupby(["Run", "Gen"]).groups.keys()
    totalPopDict["Pop"] = ["Total" for i in range(len(totalPopDict["A"]))]
    totalPop = pd.DataFrame(totalPopDict)
    totalPop = totalPop.reset_index()
    totalPop["Gen"] = totalPop["Gen"].astype('int64')
    totalPop = totalPop.sort_values(by=["Gen"])
    data = data.reset_index()
    data = data.append(totalPop, sort=False)

    data['matingSuccess'] = (data['Matings']/data['Contacts']).astype(float)
    data['misIdent'] = (data['MMContacts']/(data['Contacts']+data['MMContacts'])).astype(float)

    data.loc[data['F']==0, 'A'] = np.nan
    data.loc[data['F']==0, 'I'] = np.nan
    data.loc[data['F']==0, 'O'] = np.nan

    return data

newFile = str(len(os.listdir("../multiOutput/freqs"))+1)
print("Simulation ", newFile)
with open("../multiOutput/paramValues.tsv", "r") as paramFile:
    nameList = paramFile.readline()
    nameList = nameList.strip()
    nameList = nameList.split("\t")

with open("../multiOutput/paramValues.tsv", "a") as paramFile:
    paramValues = readParams()
    valueList = [newFile]
    for i in nameList[1:]:
        try:
            valueList.append(str(paramValues[i]))
        except KeyError:
            print(i)
            valueList.append(" ")
    valueList = "\t".join(valueList)
    print(valueList, file=paramFile)
    
data = newMultiPop.runSim("long")
with open("../multiOutput/raw/sim{}.tsv".format(newFile), "w") as dataFile:
    for line in data:
        new = []
        for i in line:
            new.append(str(i))
        print("\t".join(new), file=dataFile)
headers = data.pop(0)
data = pd.DataFrame(data, columns=headers)

data  = reshapeSingle(data)


# Plots

## Phenotype Frequency

In [4]:
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
phenoDict = {"A":"blue", "I":"green", "O":"red"}

source = ColumnDataSource(data)

output_notebook()
p = figure(title="Phenotype Frequency", x_axis_label="Generation", y_axis_label="Frequency", y_range=(0,1))
views=[]
for i in data.Pop.unique():
    views.append(CDSView(source=source,filters=[GroupFilter(column_name="Pop", group=i)]))

for phen in phenoDict.keys():
    for i in range(len(views)-1):
        name = phen+" "+str(i+1)
        p.line(x="Gen", y=phen, color=phenoDict[phen], source = source, view=views[i], line_dash=[i], legend=name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

p = figure(title="Phenotype Frequency", x_axis_label="Generation", y_axis_label="Frequency")
for phen in phenoDict.keys():
    name = phen+" "
    p.line(x="Gen", y=phen, color=phenoDict[phen], source=source, view=views[-1], legend=name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)


Loading BokehJS ...

## Population Size

In [6]:
yMin = data.loc[data["Pop"] != "Total"]["T"].min()*0.9
yMax = data.loc[data["Pop"] != "Total"]["T"].max()*1.1
p = figure(title="Population Size", x_axis_label="Generation", y_axis_label="Individuals", y_range=(yMin, yMax))
for i in range(4):
    name = str(i+1)
    p.line(x="Gen", y="T", color="black", source=source, view=views[i], line_dash=[i], legend=name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

yMin = data.loc[data["Pop"] == "Total"]["T"].min()*0.9
yMax = data.loc[data["Pop"] == "Total"]["T"].max()*1.1
p = figure(title="Total Population Size", x_axis_label="Generation", y_axis_label="Individuals", y_range=(yMin,yMax))
p.line(x="Gen", y="T", color="black", source=source, view=views[-1])
show(p)

## Fertility

In [7]:
p = figure(title="Fertility", x_axis_label="Generation", y_axis_label="Fertility", y_range=(0,1))
for i in range(len(views)-1):
    name = str(i+1)
    p.line(x="Gen", y="MalF", color="blue", source=source, view=views[i], line_dash=[i], legend="Male "+name)
    p.line(x="Gen", y="FemF", color="red", source=source, view=views[i], line_dash=[i], legend="Female "+name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

p=figure(title="Total", x_axis_label="Generation", y_axis_label="Fertility", y_range=(0,1))
p.line(x="Gen", y="MalF", color="blue", source=source, view=views[-1], legend="Male")
p.line(x="Gen", y="FemF", color="red", source=source, view=views[-1], legend="Female")
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

## Phenotype Preference
### Pre-Mating

In [8]:
p = figure(x_axis_label="Generation", y_axis_label="Preference", y_range = (0,1))
for phen in phenoDict.keys():
    for i in range(len(views)-1):
        name=phen+" "+str(i)
        p.line(x="Gen", y="pre"+phen+"Pref", color=phenoDict[phen], source=source, view=views[i], line_dash=[i], legend=name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

p = figure(title="Total", x_axis_label="Generation", y_axis_label="Preference", y_range = (0,1))
for phen in phenoDict.keys():
    name= phen+" "
    p.line(x="Gen", y="pre"+phen+"Pref", color=phenoDict[phen], source=source, view=views[-1], legend=name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

### Post-Mating

In [9]:
p = figure(x_axis_label="Generation", y_axis_label="Preference", y_range = (0,1))
for phen in phenoDict.keys():
    for i in range(len(views)-1):
        name=phen+" "+str(i)
        p.line(x="Gen", y=phen+"Pref", color=phenoDict[phen], source=source, view=views[i], line_dash=[i], legend=name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

p = figure(title="Total", x_axis_label="Generation", y_axis_label="Preference", y_range = (0,1))
for phen in phenoDict.keys():
    name= phen+" "
    p.line(x="Gen", y=phen+"Pref", color=phenoDict[phen], source=source, view=views[-1], legend=name)
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

## Interactions

In [10]:
yMin = data.loc[data["Pop"] != "Total"][["Matings", "MMContacts"]].min().min()*0.9
yMax = data.loc[data["Pop"] != "Total"]["Contacts"].max()*1.1
print(yMin, yMax)

p = figure(title="Interactions", x_axis_label="Generation", y_axis_label="Number of Interactions", y_range=(yMin, yMax))
for i in range(len(views)-1):
    p.line(x="Gen", y="Contacts", color="red", source=source, view=views[i], line_dash=[i], legend="Contacts "+str(i+1))
    p.line(x="Gen", y="MMContacts", color="blue", source=source, view=views[i], line_dash=[i], legend="MMContacts "+str(i+1))
    p.line(x="Gen", y="Matings", color="green", source=source, view=views[i], line_dash=[i], legend="Matings "+str(i+1))

p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

yMin = data.loc[data["Pop"] == "Total"][["Matings", "MMContacts"]].min().min()*0.9
yMax = data.loc[data["Pop"] == "Total"]["Contacts"].max()*1.1
p = figure(title="Interactions", x_axis_label="Generation", y_axis_label="Number of Interactions", y_range=(yMin, yMax))
p.line(x="Gen", y="Contacts", color="red", source=source, view=views[-1], legend="Contacts ")
p.line(x="Gen", y="MMContacts", color="blue", source=source, view=views[-1], legend="MMContacts ")
p.line(x="Gen", y="Matings", color="green", source=source, view=views[-1],legend="Matings ")
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)


3.6 1259.5


## Migration

In [11]:
yMin = -0.1
yMax = data.loc[data["Pop"] != "Total"]["migrations"].max()*1.1
p = figure(title="Migration", x_axis_label="Generation", y_axis_label="Number of migrants", y_range=(yMin, yMax))
for i in range(len(views)-1):
    p.line(x="Gen", y="migrations", source=source, view=views[i], line_dash=[i], legend = "Population "+str(i+1))
p.legend.location="top_left"
p.legend.click_policy="hide"
show(p)

p = figure(title="Total Migration", x_axis_label="Generation", y_axis_label="Number of migrants")
p.line(x="Gen", y="migrations", source=source, view = views[i])
show(p)
